# Reading and Exploring the Dataset

In [1]:
import gensim
import pandas as pd



In [4]:
df = pd.read_csv('/content/drive/MyDrive/imdb_master.csv', encoding= 'unicode_escape')

In [5]:
df.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [6]:
df.shape

(100000, 5)

In [7]:
df.review[0]

"Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in."

# Simple Preprocessing & Tokenization

In [9]:
review = df.review.apply(gensim.utils.simple_preprocess)

In [10]:
review.head(20)

0     [once, again, mr, costner, has, dragged, out, ...
1     [this, is, an, example, of, why, the, majority...
2     [first, of, all, hate, those, moronic, rappers...
3     [not, even, the, beatles, could, write, songs,...
4     [brass, pictures, movies, is, not, fitting, wo...
5     [funny, thing, happened, to, me, while, watchi...
6     [this, german, horror, film, has, to, be, one,...
7     [being, long, time, fan, of, japanese, film, e...
8     [tokyo, eyes, tells, of, year, old, japanese, ...
9     [wealthy, horse, ranchers, in, buenos, aires, ...
10    [cage, plays, drunk, and, gets, high, critical...
11    [first, of, all, would, like, to, say, that, a...
12    [so, tell, me, what, serious, boozer, drinks, ...
13    [big, disappointment, for, what, was, touted, ...
14    [this, film, is, absolutely, appalling, and, a...
15    [here, decidedly, average, italian, post, apoc...
16    [at, the, bottom, end, of, the, apocalypse, mo...
17    [earth, has, been, destroyed, in, nuclear,

# Training the Word2Vec Model

## Initialize the model

In [11]:
model = gensim.models.Word2Vec(window=10, sg=1, min_count=2, workers=3 )

## Build Vocabulary

In [12]:
model.build_vocab(review, progress_per=1000)

In [14]:
print(model.epochs)
model.corpus_count

5


100000

## Train the Word2Vec Model

In [15]:
model.train(review, total_examples=model.corpus_count, epochs=model.epochs)

(85733148, 112628105)

## Save the Model

In [16]:
model.save('/content/drive/MyDrive/word2vec-imdb-reviews.model')

# Finding Similar Words and Similarity between words

In [17]:
model.wv.most_similar("good")

[('decent', 0.8328449726104736),
 ('great', 0.7973397970199585),
 ('bad', 0.7820553183555603),
 ('nice', 0.7563608884811401),
 ('kamerling', 0.7506426572799683),
 ('fine', 0.7461025714874268),
 ('enoughâ', 0.7395696640014648),
 ('slughorn', 0.7383716106414795),
 ('kewl', 0.7354857325553894),
 ('alright', 0.7326892614364624)]

In [18]:
model.wv.most_similar("movie")

[('film', 0.8918426632881165),
 ('it', 0.8233982920646667),
 ('wacth', 0.7792242169380188),
 ('soooooooo', 0.7726439237594604),
 ('rubberface', 0.7683571577072144),
 ('bigtime', 0.765708327293396),
 ('this', 0.7631403207778931),
 ('strangeland', 0.7618063688278198),
 ('araf', 0.7613879442214966),
 ('woofer', 0.7599170207977295)]

In [20]:
model.wv.similarity(w1="interesting", w2="entertaining")

0.7584115

In [21]:
model.wv.similarity(w1="great", w2="good")

0.79733974

In [25]:
model.wv.most_similar("turkish")

[('kurdish', 0.699918270111084),
 ('portuguese', 0.6968694925308228),
 ('romanian', 0.6819319128990173),
 ('finnish', 0.6812542676925659),
 ('maroccan', 0.6691200137138367),
 ('norwegian', 0.6661690473556519),
 ('icelandic', 0.6631709337234497),
 ('hindi', 0.6631247401237488),
 ('tamil', 0.6617501378059387),
 ('nadu', 0.6566902995109558)]

In [29]:
model.wv.most_similar("russia")

[('germany', 0.8101766109466553),
 ('ussr', 0.7921079397201538),
 ('soviet', 0.7824720144271851),
 ('stalemate', 0.7544724345207214),
 ('europe', 0.7536591291427612),
 ('wwi', 0.7438511252403259),
 ('war', 0.7409243583679199),
 ('burma', 0.7320654392242432),
 ('provisional', 0.7285924553871155),
 ('japan', 0.7279648184776306)]

# Predicting Most Similar Words By Formula

In [48]:
vec = model['germany']-model['berlin']+model['paris']
model.most_similar([vec])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('france', 0.7103316187858582),
 ('paris', 0.6881065964698792),
 ('germany', 0.6750169992446899),
 ('italy', 0.6714487075805664),
 ('versailles', 0.65558922290802),
 ('sweden', 0.6513686180114746),
 ('coulardeau', 0.6383032202720642),
 ('europe', 0.6322596073150635),
 ('dauphine', 0.6261142492294312),
 ('sorbonne', 0.624812126159668)]

In [57]:
vec = model['king']-model['man']+model['woman']
model.most_similar([vec])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('king', 0.8320334553718567),
 ('queen', 0.7343441843986511),
 ('commoner', 0.7126818299293518),
 ('throne', 0.6929643154144287),
 ('siam', 0.6880964636802673),
 ('princess', 0.6842044591903687),
 ('prince', 0.6832098960876465),
 ('duchess', 0.6654337048530579),
 ('sheeba', 0.6621639728546143),
 ('pocahontas', 0.6585195660591125)]

In [62]:
vec = model['brad']-model['pitt']+model['depp']
model.most_similar([vec])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('depp', 0.8992573022842407),
 ('johnny', 0.6501017212867737),
 ('pearlman', 0.6361733078956604),
 ('rosanne', 0.6116749048233032),
 ('isaak', 0.6100541353225708),
 ('springsteen', 0.6037607789039612),
 ('biggs', 0.6005544066429138),
 ('hynde', 0.6003480553627014),
 ('doqui', 0.5973516702651978),
 ('quade', 0.5951790809631348)]